In [1]:
import tensorflow as tf # untuk membangun dan melatih jaringan saraf tiruan serta menjalankan komputasi numerik dengan efisien.
import numpy as np # untuk menyediakan struktur data array multidimensi yang efisien dan berbagai fungsi matematika yang kuat. 
import os # untuk berinteraksi dengan sistem operasi. 
import time # untuk mengukur waktu 

In [2]:
# untuk mengunduh file dari URL
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text)) # untuk membuat himpunan (set) karakter unik dari teks
print(f'{len(vocab)} unique characters') # mencetak jumlah karakter unik

65 unique characters


In [7]:
example_texts = ['abcdefg', 'xyz'] # mendefinisikan sebuah list yang berisi beberapa teks contoh
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8') # membagi teks menjadi karakter unicode dan untuk menentukan encoding input
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [8]:
ids_from_chars = tf.keras.layers.StringLookup( # membuat objek dan digunakan untuk mengonversi karakter menjadi ID
vocabulary=list(vocab), mask_token=None) # diberikan list karakter unik

In [10]:
ids = ids_from_chars(chars) # mengonversi karakter unicode
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [11]:
chars_from_ids = tf.keras.layers.StringLookup( # mengonversi ID kembali menjadi karakter
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None) # konversi akan dilakukan dari ID menjadi karakter

In [13]:
chars = chars_from_ids(ids) # ntuk mengonversi ID (yang disimpan dalam variabel ids) kembali menjadi karakter-karakter unicode.
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [14]:
tf.strings.reduce_join(chars, axis=-1).numpy() # menggabungkan karakter-karakter unicode menjadi sebuah string menggunakan fungsi dan mengembalikan hasilnya dalam bentuk array NumPy.

array([b'abcdefg', b'xyz'], dtype=object)

In [17]:
def text_from_ids(ids): # mengonversi ID menjadi karakter-karakter unicode.
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1) # menggabungkan karakter-karakter unicode yang dihasilkan dari konversi ID

In [18]:
# mengonversi teks dan membagi teks menjadi karakter-karakter unicode.
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [19]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids) # sumber data

In [20]:
for ids in ids_dataset.take(10): # perulangan untuk mengiterasi 10 barus dan digunakan untuk membatasai jumlah baris yang diambil
    print(chars_from_ids(ids).numpy().decode('utf-8')) # mengonversi array NumPy menjadi string menggunakan encoding UTF-8.

F
i
r
s
t
 
C
i
t
i


In [21]:
seq_length = 100

In [22]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True) # membuat sekumpulan urutan

for seq in sequences.take(1): # perulangan untuk mengiterasi melalui satu sekumpulan 
  print(chars_from_ids(seq)) # mencetak karakter unicode

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [24]:
for seq in sequences.take(5): # perualangan untuk mengiterasi melalui 5 sekumpulan urutan
    print(text_from_ids(seq).numpy()) # mengonversi setiap urutan menjadi string lalu dicetak

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [25]:
def split_input_target(sequence): # mendefinisikan fungsi yang akan membagi urutan menjadi teks input dan teks target
  input_text = sequence[:-1] # mengambil semua elemen kecuali elemen terakhir yg digunakan sebagai teks input
  target_text = sequence[1:] # mengambil semua elemen kecuali elemen terakhir yg digunakan sebagai teks target
  return input_text, target_text # mengembalikan teks input dan teks target

In [26]:
split_input_target(list("Tensorflow")) # membagi urutan menjadi teks input (baris atas) dan teks target (baris bawah )

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [27]:
dataset = sequences.map(split_input_target) # pemetaan menggunakan fungsi

In [28]:
for input_example, target_example in dataset.take(1): # perulangan untuk mengiterasi elemen dan digunakan untuk mengambil satu elemen pertama dari dataset
  print("Input :", text_from_ids(input_example).numpy()) # mencetak teks dari input_example
  print("Target:", text_from_ids(target_example).numpy()) # mencetak teks dari target_example

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [29]:
# menentukan ukuran batch yg akan digunakan dalam pelatihan
BATCH_SIZE = 64
BUFFER_SIZE = 10000 # menentukan ukuran buffer yg akan digunakan dalam pengacakan

dataset = (
    dataset
    .shuffle(BUFFER_SIZE) # metode shuffle untuk mengacak elemen dalam dataset
    .batch(BATCH_SIZE, drop_remainder=True) # untuk membagi dataset menjadi batch yg lebih kecil
    .prefetch(tf.data.experimental.AUTOTUNE)) # memuat data secara asinkron pada saat pelatihan

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [30]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [31]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units): # metode yang akan menerima 3 argumen
    super().__init__(self) # pemanggilan konstruktor dari kelas induk
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) # mengubah bilangan integer menjadi vektor
    # lapisan GRU akan mengembalikan urutan keluaran untuk setiap langkah waktu dan akan mengembalikan keadaan internal pada langkah waktu terakhir
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size) # menghasilkan output model

  def call(self, inputs, states=None, return_state=False, training=False): # mendefinisikan bagaimana model akan melakukan komputasi saat dipanggil
    x = inputs # Variabel x diinisialisasi dengan nilai inputs
    x = self.embedding(x, training=training) # Input x dilewatkan melalui lapisan embedding untuk menghasilkan vektor embedding
    # Jika states tidak diberikan (yaitu pada langkah waktu pertama), maka get_initial_state dari lapisan GRU digunakan untuk mendapatkan keadaan internal awal.
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training) # Input x dan keadaan internal states dilewatkan melalui lapisan GRU
    x = self.dense(x, training=training) # Keluaran x dari lapisan GRU dilewatkan melalui lapisan Dense untuk menghasilkan output model.

    # jika bernilai True, maka model akan mengembalikan output x dan keadaan internal states.
    if return_state:
      return x, states
    
    # jika bernilai False, maka model akan mengembalikan hanya output x.
    else:
      return x

In [32]:
model = MyModel( # membuat objek model
    vocab_size=vocab_size, # argumen yang diberikan ke parameter vocab_size saat membuat objek model. untuk menentukan jumlah kata unik dalam kamus yang akan digunakan dalam model.
    embedding_dim=embedding_dim, # argumen yang diberikan ke parameter embedding_dim saat membuat objek model. untuk menentukan dimensi vektor embedding yang akan digunakan dalam model.
    rnn_units=rnn_units) # argumen yang diberikan ke parameter rnn_units saat membuat objek model. untuk menentukan jumlah unit RNN yang akan digunakan dalam model.

In [33]:
for input_example_batch, target_example_batch in dataset.take(1): # untuk mengambil satu batch contoh dari dataset. 
    example_batch_predictions = model(input_example_batch) # model menerima input_example_batch sebagai input dan menghasilkan prediksi example_batch_predictions.
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)") # bentuk (shape) dari example_batch_predictions dicetak ke layar. 

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [34]:
# untuk mencetak ringkasan (summary) dari arsitektur model ke konsol.
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) # untuk mengambil sampel indeks dari distribusi prediksi. 
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy() # untuk menghapus dimensi yang tidak perlu dari sampled_indices. 

In [36]:
# array yang berisi indeks-indeks sampel yang diambil dari distribusi prediksi. 
sampled_indices

array([45, 54, 44, 46, 11, 46,  5, 11, 17, 58, 42, 48, 20,  0, 42,  9, 13,
       26, 23, 38,  0, 65, 23, 23, 21, 53,  3, 63,  3, 55, 65, 45, 53,  5,
       30, 26, 61, 58, 54, 50, 55, 60, 61, 21, 62, 15, 58, 54, 18, 45, 38,
       12,  5, 27, 59, 10,  8, 18, 18, 52, 24, 30, 22, 64,  2, 42, 46, 40,
        5, 19, 62, 20, 25, 18, 24, 25, 63, 24, 10, 52, 18, 14, 46, 28, 14,
       47, 10, 48, 54, 51, 15, 39, 12, 56, 44, 55, 52, 29, 23, 60])

In [37]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy()) # untuk mengonversi tensor input_example_batch[0] menjadi teks.
print() # mencetak baris kosong sebagai pemisah antara teks input dan prediksi karakter selanjutnya.
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy()) # untuk mengonversi array sampled_indices menjadi teks.

Input:
 b" seem tedious,\nI'll tell thee what befell me on a day\nIn this self-place where now we mean to stand."

Next Char Predictions:
 b'foeg:g&:DsciG[UNK]c.?MJY[UNK]zJJHn!x!pzfn&QMvsokpuvHwBsoEfY;&Nt3-EEmKQIy cga&FwGLEKLxK3mEAgOAh3iolBZ;qepmPJu'


In [39]:
# mendefinisikan fungsi kerugian (loss function) yang akan digunakan dalam pelatihan model.
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [40]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions) # fungsi loss digunakan untuk menghitung kerugian antara target_example_batch dan example_batch_predictions.
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") # mencetak bentuk (shape) dari example_batch_predictions.
print("Mean loss:        ", example_batch_mean_loss) # mencetak nilai kerugian rata-rata example_batch_mean_loss.

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.190308, shape=(), dtype=float32)


In [41]:
tf.exp(example_batch_mean_loss).numpy() # menghitung eksponen dari nilai example_batch_mean_loss.

66.04314

In [42]:
model.compile(optimizer='adam', loss=loss) # Argumen optimizer digunakan untuk menentukan optimizer yang akan digunakan saat melatih model. Argumen loss digunakan untuk menentukan fungsi kerugian yang akan digunakan dalam pelatihan model.

In [43]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
# objek ModelCheckpoint yang digunakan sebagai callback selama pelatihan model. Objek ini akan menyimpan checkpoint selama pelatihan. 
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [44]:
EPOCHS = 20 # EPOCHS diatur sebagai 20

In [45]:
# untuk melatih model dengan dataset yang diberikan, menggunakan jumlah epoch sebanyak nilai EPOCHS yang telah ditentukan sebelumnya, dan menggunakan callback checkpoint_callback yang telah dibuat sebelumnya. 
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 965s 6s/step - loss: 2.7189
Epoch 2/20
172/172 [==============================] - 920s 5s/step - loss: 1.9914
Epoch 3/20
172/172 [==============================] - 924s 5s/step - loss: 1.7106
Epoch 4/20
172/172 [==============================] - 961s 6s/step - loss: 1.5479
Epoch 5/20
172/172 [==============================] - 962s 6s/step - loss: 1.4496
Epoch 6/20
172/172 [==============================] - 968s 6s/step - loss: 1.3814
Epoch 7/20
172/172 [==============================] - 959s 6s/step - loss: 1.3292
Epoch 8/20
172/172 [==============================] - 934s 5s/step - loss: 1.2848
Epoch 9/20
172/172 [==============================] - 897s 5s/step - loss: 1.2443
Epoch 10/20
172/172 [==============================] - 922s 5s/step - loss: 1.2042
Epoch 11/20
172/172 [==============================] - 908s 5s/step - loss: 1.1651
Epoch 12/20
172/172 [==============================] - 910s 5s/step - loss: 1.1244
Epoch 13/20
1

In [46]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0): # untuk menginisialisasi objek OneStep dengan model yang diberikan
    super().__init__()
    self.temperature = temperature #  atribut yang menyimpan suhu yang digunakan untuk sampling stokastik
    self.model = model # atribut yang menyimpan model yang digunakan untuk generasi teks.
    # atribut yang menyimpan fungsi konversi karakter ke ID dan ID ke karakter
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None] # ID yang digunakan untuk mengabaikan token "[UNK]" (unknown) saat generasi teks.
    sparse_mask = tf.SparseTensor( # masker yang diterapkan pada prediksi untuk mencegah token "[UNK]" dari dihasilkan dalam generasi teks. 
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask) #  atribut yang menyimpan masker prediksi sebagai tensor padat (dense tensor) untuk digunakan dalam proses generasi teks.

  @tf.function # mengubah metode generate_one_step menjadi graf TensorFlow yang dapat dipercepat untuk meningkatkan efisiensi.
  def generate_one_step(self, inputs, states=None): # untuk melakukan generasi teks satu langkah pada suatu waktu.
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8') #  mengubah input teks menjadi urutan karakter Unicode.
    input_ids = self.ids_from_chars(input_chars).to_tensor() # Hasil ID token kemudian dikonversi menjadi tensor.

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [47]:
# Objek ini dibuat dengan menggunakan model yang diberikan, serta fungsi konversi karakter ke ID (chars_from_ids) dan konversi ID ke karakter (ids_from_chars) yang telah didefinisikan sebelumnya.
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [48]:
start = time.time() # memulai penghitungan waktu untuk mengukur berapa lama waktu yang dibutuhkan 
states = None # keadaan awal model yang akan digunakan dalam generasi teks. 
next_char = tf.constant(['ROMEO:']) # karakter awal yang digunakan sebagai input untuk memulai generasi teks.
result = [next_char] # daftar yang akan menyimpan urutan karakter yang dihasilkan selama generasi teks.

for n in range(1000): # loop yang akan melakukan generasi teks sebanyak 1000 langkah.
  next_char, states = one_step_model.generate_one_step(next_char, states=states) # generasi teks
  result.append(next_char) # Karakter yang diprediksi selanjutnya ditambahkan ke dalam daftar result.

result = tf.strings.join(result) # untuk menggabungkan semua karakter dalam daftar result menjadi satu teks yang lengkap.
end = time.time() # menghentikan penghitungan waktu setelah generasi teks selesai.
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80) # mencetak hasil generasi teks.
print('\nRun time:', end - start) # mencetak berapa lama waktu yang dibutuhkan untuk melakukan generasi teks.

ROMEO:
What, ha? hasty in Vienna.

GLOUCESTER:
In sort and ne'er then had lost his mennor.

BALTHASAR:
At what hat hath done all thy sexsidicy,
Hast whom thy embracements mine enemies;
And, in some reprehend honey-athoraxt, they deny the earth,
Our virtues had ever than he is,
And they believe, that thou entire, are they
not thy amile that must be, 'dial foul hundred prosperate,
To step bowels, that, if thou deartiness
For serves import, what lies this man that calls
A Romanoble to condition much:
There's not as highness that must bear me to
Left this most now live the single toen,
And it do I exple to Englander, and his
short shall be her womanies; it is in
Your back o' the law in the office; or towards
And pieped out the world's vows; and, as they were,
Brought down to many mimes; onless, ourselves,
Dived with distressy-budly lives;
For time shall ward my country's kinsment hangled.
'Tis thought the more revave? I have conspired his head;
This bastard love, or shall well have heads,


In [49]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nFor beauty is fam, and ask you hence;\nOur--ask, Bagab,--anster, how many locks.\n\nSecond Conspirator:\nSo peeves.\n\nLEONTES:\nDo you not resemp\nTill I came unto these weeds are pawed.\nIs it the heaven, that all the world as his son abuse,\nShe might shall dispropht their fulless of either'd son;\nFor whose corrupt away us no agree?\nYou\ngo about it, I will answer that which stain he work\nThat scorn our subjects may prove a Christian love,\nAnd get your good opposer, suffer as the does of tume\nUnto mine ears against the day:\nO Gresces where he hath a found his son't,\nwould not the abstance: take that justice,\nBut that he wakens it up with regreemory,\nAnd please the hopeful accusden, who\nrather set down out the rest:\nAppeal all your person be hurl'd,\nLandary, he should to hold me how\nThat bears most part in a dearer deed desert:\nif, not, thus. Ky'll that alike your people\nUntil Gon' seen him that is not high'd to seat it.\n\nDUKE OF AUMERLE:\nWith 

In [50]:
tf.saved_model.save(one_step_model, 'one_step') # untuk menyimpan objek one_step_model ke dalam format SavedModel. 
one_step_reloaded = tf.saved_model.load('one_step') # untuk memuat kembali model yang telah disimpan sebelumnya. 

In [51]:
states = None # menginisialisasi keadaan model states menjadi None.
next_char = tf.constant(['ROMEO:']) # menentukan karakter awal yang akan digunakan sebagai input untuk memulai generasi teks.
result = [next_char] # membuat daftar result yang akan menyimpan urutan karakter yang dihasilkan selama generasi teks.

for n in range(100): # loop sebanyak 100 langkah untuk melakukan generasi teks.
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states) # generasi teks 
  result.append(next_char) # Karakter yang diprediksi selanjutnya ditambahkan ke dalam daftar result.

print(tf.strings.join(result)[0].numpy().decode("utf-8")) # mencetak hasil generasi teks.

ROMEO:
O, what is it?

GLOUCESTER:
Here!
Saint whom here I mistrust now what you sought.

GLOUCESTER:
How!
